# Types and Dispatch in Julia

Julia is built around types.

Software architectures in Julia are built around good use of the type system.

# Abstract vs concrete types

*Concrete types* are the types of objects. They specify the data structure of an object.

*Abstract types* cannot be instantiated. They define sets of related concrete types (their descendants) by their behavior.

In [1]:
typeof(3)

Int64

In [2]:
typeof(3.0)

Float64

In [3]:
isconcretetype(Float64)

true

In [4]:
isabstracttype(Number)

true

In [5]:
isabstracttype(Real)

true

### Duck typing

A `Number` is some type that can do things like `+`,`-`,`*`, and `/`. In this category we have things like `Float64` and `Int32`.

An `AbstractArray` is a type that can be indexed like `A[i]`. An `AbstractArray` may be mutable, meaning it can be set: `A[i]=v`.

### Inspecting the type tree

In [18]:
supertype(Float64)

AbstractFloat

In [19]:
supertype(AbstractFloat)

Real

In [20]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [4]:
supertype(Real)

Number

In [5]:
supertype(Number)

Any

Everything is a subtype of Any

In [6]:
Number <: Any

true

In [7]:
Float64 <: Any

true

In [8]:
Int32 <: Any

true

In [9]:
Int32 <: String

false

There is also `isa` for objects:

In [10]:
3.0 isa Float64

true

In [11]:
3 isa Float64

false

We define a function that, given a concrete type `T`, prints the single branch of the type tree that leads from the top node `Any` to the leave `T`.

In [22]:
function show_supertypes(T) 
 print(T)
 while T != Any 
     T = supertype(T) 
     print(" <: ", T) 
 end 
end

show_supertypes (generic function with 1 method)

In [23]:
show_supertypes(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any

In [24]:
show_supertypes(String)

String <: AbstractString <: Any

Let's extract a bunch of branches

In [26]:
function show_subtypetree(T, level=1, indent=4)
   level == 1 && println(T)
   for s in subtypes(T)
     println(join(fill(" ", level * indent)) * string(s))
     show_subtypetree(s, level+1, indent)
   end
end

show_subtypetree (generic function with 3 methods)

In [27]:
show_subtypetree(Number)

Number
    Complex
    Real
        AbstractFloat
            BigFloat
            Float16
            Float32
            Float64
        AbstractIrrational
            Irrational
        Integer
            Bool
            Signed
                BigInt
                Int128
                Int16
                Int32
                Int64
                Int8
            Unsigned
                UInt128
                UInt16
                UInt32
                UInt64
                UInt8
        Rational


Note that concrete types are the leaves of the type tree.

Abstract types are nodes in the type graph.

# Functions, Methods, and Dispatch

Let's define a *function* that calculates the absolute value of a number (like Julias `abs` already does).

How would we practically calculate the absolute values of the numbers $-4.32$ and $1.0 + 1.0i$?

Presumably:
* Real number: "Drop the sign." => `myabs(-4.32) = 4.32`
* Complex number: "Square root of z times the complex conjugate of z." => `myabs(1.0 + 1.0im) = sqrt(2) ≈ 1.414`

We see that the *methods* that we use depend on the type of the number.

While the single **function** represents the *what* ("calculate the absolute value"), there might be different **methods** describing the *how*.

We can use the `::` operator to annotate function arguments with types and define different methods.

In [1]:
myabs(x::Float64) = sign(x) * x

myabs (generic function with 1 method)

In [2]:
myabs(-4.32)

4.32

In [3]:
myabs(1.0 + 1.0im)

MethodError: MethodError: no method matching myabs(::Complex{Float64})
Closest candidates are:
  myabs(!Matched::Float64) at In[1]:1

In [4]:
myabs(z::ComplexF64) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [5]:
myabs(1.0 + 1.0im)

1.4142135623730951

In [6]:
methods(myabs)

# 2 methods for generic function "myabs":
[1] myabs(z::Complex{Float64}) in Main at In[4]:1
[2] myabs(x::Float64) in Main at In[1]:1

One can check which particular method is being used through the `@which` macro.

In [7]:
@which myabs(-4.32)

myabs(x::Float64) in Main at In[1]:1

In [8]:
@which myabs(1.0 + 1.0im)

myabs(z::Complex{Float64}) in Main at In[4]:1

Note that we should better loosen our type restrictions:

In [9]:
myabs(-3)

MethodError: MethodError: no method matching myabs(::Int64)
Closest candidates are:
  myabs(!Matched::Complex{Float64}) at In[4]:1
  myabs(!Matched::Float64) at In[1]:1

In [10]:
myabs(1 + 1im)

MethodError: MethodError: no method matching myabs(::Complex{Int64})
Closest candidates are:
  myabs(!Matched::Complex{Float64}) at In[4]:1
  myabs(!Matched::Float64) at In[1]:1

In [11]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 4 methods)

In [12]:
myabs(-3)

3

# Multiple Dispatch

Julia's dispatch mechanism always chooses the *most specific method* for the given input types.

In [13]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [14]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[13]:5
[2] f(a::Number, b::Number) in Main at In[13]:2
[3] f(a::Number, b) in Main at In[13]:3
[4] f(a, b::Number) in Main at In[13]:4
[5] f(a, b) in Main at In[13]:1

In [15]:
f(1.5, 2)

"a and b are both numbers"

In [16]:
f(1, "Köln!")

"a is a number"

In [17]:
f(1, 2)

"a and b are both integers"

In [18]:
f("Hello", "World!")

"fallback"

In [19]:
@which f(1, 2)

f(a::Integer, b::Integer) in Main at In[13]:5

In [20]:
@which f(1, "Köln!")

f(a::Number, b) in Main at In[13]:3

In [21]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\s

In [22]:
@which true + false

+(x::Bool, y::Bool) in Base at bool.jl:104

In [23]:
@which "Hello"*"World!"

*(s1::Union{AbstractChar, AbstractString}, ss::Union{AbstractChar, AbstractString}...) in Base at strings/basic.jl:229

Julia's standard functions are not special by any means.

We can easily modify or add methods to them as well.

In [2]:
import Base: + # we have to import functions to override/extend them
+(x::String, y::String) = x * " " * y

+ (generic function with 164 methods)

In [3]:
"Kölle" + "Alaaf"

"Kölle Alaaf"

Any function based on the `+` operation can now handle `String`s as well.

In [5]:
sum(["This", "works", "although", "we", "never", "touched", "sum!"])

"This works although we never touched sum!"

It happens rarely, but it can happen that there is no unique most specific method:

In [19]:
f(x::Int, y::Any) = println("int")
f(x::Any, y::String) = println("string")
f(3, "test")

MethodError: MethodError: f(::Int64, ::String) is ambiguous. Candidates:
  f(x, y::String) in Main at In[15]:2
  f(x::Int64, y) in Main at In[19]:1
Possible fix, define
  f(::Int64, ::String)

# Parametric types

Types can have *type parameters*. The most prominent example is Julia's array type.

In [29]:
M = rand(2,2)

2×2 Array{Float64,2}:
 0.832619  0.229362
 0.830405  0.500011

In [30]:
typeof(M)

Array{Float64,2}

Here, `Array` is a parametric array datatype. Its type parameters `Float64` and `2` indicate the type of the element the array can hold and and its dimensionality. Hence, we have a matrix of floating point numbers.

This generalizes as expected. Here, a matrix of `String`s:

In [43]:
M = fill("Cologne", 2,2)

2×2 Array{String,2}:
 "Cologne"  "Cologne"
 "Cologne"  "Cologne"

In [45]:
eltype(M)

String

We can also nest parametric types. This is a vector of matrices of `Float64`s.

In [37]:
v = [rand(2,2) for i in 1:3]

3-element Array{Array{Float64,2},1}:
 [0.341835 0.552818; 0.376563 0.264724]
 [0.326616 0.857752; 0.564207 0.185871]
 [0.644797 0.190454; 0.765222 0.907248]

In [38]:
eltype(v)

Array{Float64,2}

Since vectors and matrices pop up so frequently, Julia has a nice shortcut type alias for them.

In [46]:
Vector{Float64} === Array{Float64, 1}

true

In [47]:
Matrix{Float64} === Array{Float64, 2}

true

Another example of a parametric type is the `Tuple`.

In [50]:
(1,2.0,"3")

(1, 2.0, "3")

In [51]:
typeof((1,2.0,"3"))

Tuple{Int64,Float64,String}

### `UnionAll` types and `where`

Note that parametric types have the following (somewhat counterintuitive at first) property

In [57]:
Vector{Float64} <: Vector{Real}

false

although we have

In [58]:
Float64 <: Real

true

The reason is that `Vector{Real}` is a concrete type - it describes a vector of values that individually have a type `T <: Real`  - and concrete types don't have subtypes.

In [9]:
isconcretetype(Vector{Real})

true

In [10]:
Real[1, 2.2, 13f0]

3-element Array{Real,1}:
  1    
  2.2  
 13.0f0

What we actually *mean* is

In [80]:
Vector{Float64} <: Vector{T} where T<:Real

true

Here, `Vector{T} where T <: Real` describes the *set* of concrete `Vector` types whose elements are of any single type `T` that is a subtype of `Real`.

In [81]:
Vector{Int64} <: Vector{T} where T<:Real

true

In [82]:
Vector{Real} <: Vector{T} where T<:Real

true

Using this notation, our `Vector{Real}` from above can more explicitly be written as `Vector{T where T<:Real}`

In [13]:
Vector{Real} === Vector{T where T<:Real}

true

### Type parameters in function signatures

In [104]:
h(x::Integer) = typeof(x)

h (generic function with 2 methods)

In [105]:
h(x::T) where T = T

h (generic function with 3 methods)

**Quick exercise**: Write a single-argument function that takes any real matrix as input and returns the element type of the matrix.

**Solution:**
<details>
  <summary>Click to reveal</summary>
<br>
    
```julia
g(x::Matrix{T}) where T<:Real = T
```

or alternatively

```julia
g(x::Matrix{<:Real}) = eltype(x)
```
</details>

**Test:**

In [102]:
g(rand(Float32,2,2))

Float32

In [100]:
g(rand(Int16,2,2))

Int16

# "Diagonal" dispatch

In [133]:
d(x::T, y::T) where T = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [134]:
d(3, 4)

"same type"

In [135]:
d(3.0, 1.0)

"same type"

In [137]:
d(1, 4.2)

"different types"

# `Union` types

In [12]:
mytypes = Union{Float64, Int64}

Union{Float64, Int64}

In [13]:
Float64 <: mytypes

true

In [14]:
Int64 <: mytypes

true

In [15]:
Float32 <: mytypes

false

# Duck typing examples

### `UnitRange`

In [171]:
x = 1:30

1:30

In [172]:
typeof(x)

UnitRange{Int64}

In [110]:
typeof(x) <: AbstractArray

true

Because it is a subtype of `AbstractArray` we can do array-like things with it (it should basically behave like an array!)

In [174]:
x[3]

3

In [176]:
size(x)

(30,)

In [177]:
eltype(x)

Int64

However, it's not implemented like a regular `Array` at all.

In fact, it's just two numbers! We can see this by looking at it's fields:

In [93]:
fieldnames(typeof(x))

(:start, :stop)

or just by inspecting the source code

In [183]:
@which UnitRange(1, 30)

(::Type{UnitRange})(start::T, stop::T) where T<:Real in Base at range.jl:276

It is an `immutable` type which just holds the start and stop values.

This means that indexing, `A[i]`, is not just a look-up but a (small) function (try `@which getindex(x, 4)`).

What's nice about this is that we can use it in calculations and no array, containing the numbers from 1 to 30, is ever created.

Allocating memory is typically costly.

In [187]:
@time collect(1:10000000);

  0.087493 seconds (7 allocations: 76.294 MiB, 72.13% gc time)


But creating an immutable type of two numbers is essentially free, no matter what those two numbers are:

In [190]:
@time 1:10000000;

  0.000002 seconds (5 allocations: 192 bytes)


Yet, in code they *act* the same way.

### `UniformScaling` operator

Another great example is the `UniformScaling` operator. It automatically gets loaded into scope when you do `using LinearAlgebra` and has the name `I`.

In [191]:
using LinearAlgebra

In [192]:
I

UniformScaling{Bool}
true*I

In [193]:
?I

search: I IO In if Int in im Inf isa Int8 inv Int64 Int32 Int16 imag Inf64 Inf32



```
I
```

An object of type [`UniformScaling`](@ref), representing an identity matrix of any size.

# Examples

```jldoctest
julia> fill(1, (5,6)) * I == fill(1, (5,6))
true

julia> [1 2im 3; 1im 2 3] * I
2×3 Array{Complex{Int64},2}:
 1+0im  0+2im  3+0im
 0+1im  2+0im  3+0im
```


Although it never actually materializes a full identity matrix it behaves like one!

In [197]:
M = rand(1:10, 2,2)

2×2 Array{Int64,2}:
 2  6
 9  7

In [198]:
I * M

2×2 Array{Int64,2}:
 2  6
 9  7

In [199]:
M + I

2×2 Array{Int64,2}:
 3  6
 9  8

Hence, we can calculate things, say `A-b*I`, without ever allocating a dense identity matrix, which would take up $\mathcal{O}(n^2)$ memory.

Let's benchmark the performance difference!

In [260]:
N = 1000
M = rand(N,N)
fullI = Matrix(1.0I, N,N) # alternatively but slower, diagm(0 => [1.0, 1.0])

1000×1000 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [270]:
fast(M) = M + 3*I

fast (generic function with 1 method)

In [271]:
slow(M, fullI) = M + 3*fullI

slow (generic function with 1 method)

In [276]:
function slower(M)
    fullI = Matrix(1.0I, size(M)...)
    M + 3*fullI
end

slower (generic function with 1 method)

In [284]:
@time fast(M);
@time slow(M, fullI);
@time slower(M);

  0.002416 seconds (6 allocations: 7.630 MiB)
  0.010202 seconds (8 allocations: 15.259 MiB, 33.68% gc time)
  0.011036 seconds (10 allocations: 22.889 MiB, 21.01% gc time)


Note that benchmarks with `@time` are almost always flawed. One should generally use `@btime` from the package `BenchmarkTools` as we will explore later in the workshop.

# Other types

* Union types: `Union{Float64, Int32}`
* [Bitstypes](https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/#man-bits-types-1) (check with `isbits(x)`, `isbitstype(T)`)
* [Value types](https://docs.julialang.org/en/v1/manual/types/#%22Value-types%22-1) (allows dispatch on values)

See https://docs.julialang.org/en/latest/manual/types/ for more.

# Extra: slurping and splatting

In [291]:
f(x...) = println(x) # slurping

f (generic function with 3 methods)

In [286]:
f(3, 1.2, "Carsten")

(3, 1.2, "Carsten")


In [292]:
g(x::Vector) = +(x...) # splat vector into addition operation

g (generic function with 4 methods)

In [293]:
g([1,2,3])

6

# Core messages of this Notebook

* **Concrete types** describe data structures, i.e. concrete implementations.
* **Abstract types** define the kind of a thing (What is it? What can I do with it?), i.e. an informal interface. This is also known as **duck-typing**.
* A **function** (the what) can have multiple **methods** (the how).
* **Multiple dispatch**: Julia selects the method to run based on the types of all input arguments and chooses the most specialized one.
* Types can have parameters, i.e. `Vector{Float64}`. We can use the notation `T where T<:SomeSuperType` to address *sets* of types.